In [1]:
import numpy as np
import pandas as pd
import torch
from pyabc import Distribution, RV
from sbi.utils import BoxUniform

c:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.utils import set_seed
from src.inference import SBIEngine
from models.epidemic_models import simulate_seir  

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


In [3]:
# 1. Global Setup
set_seed(0) # Fix seed for reproducibility
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

[#] Seed has been fixed to: 0
Using device: cpu


In [ ]:
# 2. Define Priors (Two versions for pyabc and sbi compatibility)
# pyabc version (loc, scale) -> [0.2, 1.5], [0.1, 0.5], [0.05, 0.40]
pyabc_prior = Distribution(
    beta=RV("uniform", 0.2, 1.3),
    kappa=RV("uniform", 0.1, 0.4),
    gamma=RV("uniform", 0.05, 0.35)
)

# sbi version (low, high)
sbi_prior = BoxUniform(
    low=torch.tensor([0.2, 0.1, 0.05]),
    high=torch.tensor([1.5, 0.5, 0.40])
)

In [5]:
# 3. Define Simulator Wrappers
def simulator(params):
    p = [params['beta'], params['kappa'], params['gamma']]
    output = simulate_seir(p, [99990, 0, 10, 0], duration=100)
    return output

In [6]:
def generate_training_data(num_sims=10000):
    """Generates training dataset for standard NPE and NPE-LSTM"""
    print(f"[*] Generating {num_sims} simulations for NPE training...")
    thetas = sbi_prior.sample((num_sims,))
    xs = []
    for t in thetas:
        data = simulate_seir(t.numpy(), [99990, 0, 10, 0], duration=100)
        xs.append(torch.tensor(data, dtype=torch.float32))
    return thetas, torch.stack(xs)

In [7]:
# 4. Prepare Observed Data (Ground Truth)
true_params = [0.8, 0.3, 0.1]
observed_raw = simulate_seir(true_params, [99990, 0, 10, 0], duration=100)
obs_dict = {"data": observed_raw}

In [8]:
engine = SBIEngine()

In [9]:
abc_samples = engine.run_abc(
    obs_data=obs_dict,
    prior=pyabc_prior,
    simulator_func=simulator,
    num_simulations=1000, 
    population_size=100,
    num_samples=10000
)

[*] Running SMC-ABC...


ABC.History INFO: Start <ABCSMC id=1, start_time=2026-02-03 16:22:21>
ABC INFO: Calibration sample t = -1.
ABC INFO: t: 0, eps: 1.35598722e+04.
ABC INFO: Accepted: 100 / 357 = 2.8011e-01, ESS: 1.0000e+02.
ABC INFO: t: 1, eps: 3.55534378e+03.
ABC INFO: Accepted: 100 / 530 = 1.8868e-01, ESS: 7.6360e+01.
ABC INFO: t: 2, eps: 1.59044881e+03.
ABC INFO: Accepted: 100 / 684 = 1.4620e-01, ESS: 4.5645e+01.
ABC INFO: Stop: Total simulations budget.
ABC.History INFO: Done <ABCSMC id=1, duration=0:00:03.915766, end_time=2026-02-03 16:22:25>


In [13]:
abc_samples

name,beta,gamma,kappa
id,,,
298,0.783400,0.117561,0.353310
261,0.675679,0.108252,0.471968
250,0.755180,0.160098,0.418195
204,0.629427,0.047169,0.396211
292,0.638646,0.114563,0.500797
...,...,...,...
205,0.805238,0.126041,0.321500
239,0.879711,0.182636,0.356947
204,0.557212,0.088804,0.510447


In [11]:
# ==========================================================
# METHOD 4: PNPE (Preconditioned NPE)
# ==========================================================
# PNPE Stage 1 uses ABC, then Stage 2 trains a refined NPE
pnpe_post, pnpe_samples = engine.run_pnpe(
    obs_data=observed_raw,
    pyabc_prior=pyabc_prior,
    sbi_prior=sbi_prior,
    simulator_func=simulator,
    num_simulations=1000,
    num_samples=10000, 
    batch_size=64
)

ABC.History INFO: Start <ABCSMC id=1, start_time=2026-02-03 16:22:40>
ABC INFO: Calibration sample t = -1.


[*] PNPE Stage 1: ABC Preconditioning...
[*] Running SMC-ABC...


ABC INFO: t: 0, eps: 1.51894131e+04.
ABC INFO: Accepted: 100 / 302 = 3.3113e-01, ESS: 1.0000e+02.
ABC INFO: t: 1, eps: 3.35711937e+03.
ABC INFO: Accepted: 100 / 420 = 2.3810e-01, ESS: 7.5285e+01.
ABC INFO: Stop: Total simulations budget.
ABC.History INFO: Done <ABCSMC id=1, duration=0:00:01.881776, end_time=2026-02-03 16:22:42>


[*] PNPE Stage 2: Training NPE with preconditioned samples...
[*] Running NPE (use_lstm=True) with batch size 64...
 Neural network successfully converged after 98 epochs.
        -------------------------
        ||||| ROUND 1 STATS |||||:
        -------------------------
        Epochs trained: 98
        Best validation performance: -4.5658
        -------------------------
        


11610it [00:00, 79323.06it/s]            


In [ ]:
# ==========================================================
# METHOD 2: Standard NPE (No LSTM embedding)
# ==========================================================
npe_post, npe_samples = engine.run_npe(
    obs_data=observed_raw,
    prior=sbi_prior,
    thetas=thetas_train,
    xs=xs_train,
    use_lstm=False,
    num_samples=10000
)

In [12]:
# ==========================================================
# METHOD 2: Standard NPE (No LSTM embedding)
# ==========================================================
npe_post, npe_samples = engine.run_npe(
    obs_data=observed_raw,
    prior=sbi_prior,
    thetas=thetas_train,
    xs=xs_train,
    use_lstm=False,
    num_samples=10000
)

NameError: name 'thetas_train' is not defined